# StackAI Vector Database Exploration

Interactive notebook for testing the StackAI vector database API.

**Setup (Local Development):**
1. Start the server: `make start`
2. Seed test data: `python scripts/seed_data.py --library all`
3. Run the cells below

**Setup (Docker):**
1. Start the container: `docker compose up --build`
2. Seed test data: `docker compose exec api python scripts/seed_data.py --library all`
3. Run the cells below

The API is available at `http://localhost:8000` in both cases.

In [ ]:
import sys
sys.path.insert(0, "..")

from client import StackAIClient

# Create client instance
client = StackAIClient()

# Convenience aliases for common operations
search = client.print_search
list_libraries = client.print_libraries
list_documents = client.print_documents
list_chunks = client.print_chunks
health_check = client.print_health

print("Client loaded. Available functions: search(), list_libraries(), list_documents(), list_chunks(), health_check()")
print("Direct API access via: client.create_library(), client.search(), etc.")

## Quick Start

In [2]:
health_check()

Server is running


In [ ]:
list_libraries()

## Search

Available libraries (after seeding with `seed_data.py --library all`):
- Library 1: Recipe Collection
- Library 2: Support Knowledge Base  
- Library 3: Product Manuals

Use `list_libraries()` to see current library IDs and names.

In [ ]:
# Search recipes library (ID=1 after seeding)
search("How do I make a creamy pasta sauce?", library_id=1)

In [ ]:
# Search support library (ID=2 after seeding)
search("How do I reset my password?", library_id=2)

In [ ]:
# Search products library (ID=3 after seeding)
search("bluetooth pairing", library_id=3, k=5)

In [ ]:
# Try your own query
search("chicken curry recipe", library_id=1, k=5)

## Browse Data

In [ ]:
# List documents in recipes library (ID=1)
list_documents(1)

In [ ]:
# List chunks in first document (ID=1)
# After seeding, document 1 is usually "Spaghetti Carbonara"
list_chunks(1)

## Manual API Calls

For more control, use the client directly:

In [ ]:
# Raw API call example - using client methods
# Note: search() takes library_id first, then query
result = client.search(library_id=1, query="baking cookies", k=2)
result

## Testing Deletes

Test that deleting chunks/documents/libraries properly removes them from both storage and the search index.

In [ ]:
# Create a test library for delete testing
lib = client.create_library("Delete Test Library")
delete_test_lib_id = lib["id"]
print(f"Created library (ID={delete_test_lib_id})")

# Create a document
doc = client.create_document(delete_test_lib_id, "Test Document")
delete_test_doc_id = doc["id"]
print(f"Created document (ID={delete_test_doc_id})")

# Create chunks - IDs are now server-generated
chunks = [
    {"text": "The quick brown fox jumps over the lazy dog"},
    {"text": "Machine learning is a subset of artificial intelligence"},
    {"text": "Python is a popular programming language"},
]
result = client.create_chunks_batch(delete_test_doc_id, chunks)
print(f"Created {result.get('created_count', 0)} chunks")

# Store chunk IDs for delete tests
chunk_ids = [c["id"] for c in result.get("chunks", [])]
print(f"Chunk IDs: {chunk_ids}")

In [ ]:
# Verify search works - search for "programming"
print("=== BEFORE DELETE ===")
search("programming language", library_id=delete_test_lib_id, k=3)

In [ ]:
# Delete single chunk (the Python programming one - last in our list)
python_chunk_id = chunk_ids[2]  # "Python is a popular programming language"
client.delete_chunk(python_chunk_id)
print(f"Deleted chunk {python_chunk_id}")

# Verify it's gone from search results
print("\n=== AFTER DELETING PYTHON CHUNK ===")
search("programming language", library_id=delete_test_lib_id, k=3)
# Should NOT find the Python chunk anymore

In [ ]:
# Create another document with chunks to test document deletion
doc2 = client.create_document(delete_test_lib_id, "Second Test Document")
delete_test_doc2_id = doc2["id"]
print(f"Created document2 (ID={delete_test_doc2_id})")

chunks2 = [
    {"text": "Cats are popular pets around the world"},
    {"text": "Dogs are known as man's best friend"},
]
client.create_chunks_batch(delete_test_doc2_id, chunks2)
print("Created chunks")

# Verify we can find pet content
print("\n=== BEFORE DOCUMENT DELETE ===")
search("pets and animals", library_id=delete_test_lib_id, k=5)

In [ ]:
# Delete the document (should cascade delete its chunks from index too)
client.delete_document(delete_test_lib_id, delete_test_doc2_id)
print(f"Deleted document2 (ID={delete_test_doc2_id})")

# Verify pet chunks are gone from search
print("\n=== AFTER DOCUMENT DELETE ===")
search("pets and animals", library_id=delete_test_lib_id, k=5)
# Should NOT find cats or dogs chunks anymore

In [ ]:
# Test library deletion (deletes everything including index file)
client.delete_library(delete_test_lib_id)
print(f"Deleted library (ID={delete_test_lib_id})")

# Verify library is gone
import httpx
try:
    client.get_library(delete_test_lib_id)
    print("ERROR: Library still exists!")
except httpx.HTTPStatusError as e:
    print(f"Get deleted library: {e.response.status_code} (expected 404)")

# Verify search fails gracefully
try:
    client.search(delete_test_lib_id, "test", k=3)
    print("ERROR: Search should have failed!")
except httpx.HTTPStatusError as e:
    print(f"Search deleted library: {e.response.status_code} (expected 404)")

print("\nDelete tests complete!")

## Testing Index Persistence

Test that indexes survive server restarts.

**Manual test:**
1. Run the cell below to create test data
2. Restart the server (Ctrl+C, then `make start`)
3. Run the verification cell to confirm search still works

In [ ]:
# Step 1: Create test data for persistence test
lib = client.create_library("Persistence Test Library")
persist_test_lib_id = lib["id"]
print(f"Created library (ID={persist_test_lib_id})")

doc = client.create_document(persist_test_lib_id, "Persistence Test Doc")
persist_test_doc_id = doc["id"]
print(f"Created document (ID={persist_test_doc_id})")

persist_chunks = [
    {"text": "This chunk should survive a server restart"},
    {"text": "Index persistence means the search index is saved to disk"},
]
client.create_chunks_batch(persist_test_doc_id, persist_chunks)
print("Created chunks")

print(f"\nTest data created with library_id={persist_test_lib_id}")
print("Now restart the server and run the next cell.")

In [ ]:
# Step 2: Verify search still works after restart
# (Run this AFTER restarting the server)
# Note: You need to know the library ID from the previous step

# Find the persistence test library
libs = client.get_libraries()
persist_lib = next((l for l in libs if l["name"] == "Persistence Test Library"), None)

if persist_lib:
    persist_test_lib_id = persist_lib["id"]
    print(f"Found persistence test library (ID={persist_test_lib_id})")
    print("\nChecking if search works after restart...")
    search("server restart persistence", library_id=persist_test_lib_id, k=3)
    print("\nIf you see results above, index persistence is working!")
else:
    print("Persistence test library not found. Run the previous cell first.")

In [ ]:
# Cleanup: Delete persistence test library
# Find and delete the persistence test library
libs = client.get_libraries()
persist_lib = next((l for l in libs if l["name"] == "Persistence Test Library"), None)

if persist_lib:
    client.delete_library(persist_lib["id"])
    print(f"Deleted persistence test library (ID={persist_lib['id']})")
else:
    print("Persistence test library not found (already cleaned up?)")